In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import metrics
import matplotlib.pyplot as plt
from keras.utils import plot_model

from sklearn.metrics import r2_score

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

from scikeras.wrappers import KerasRegressor

import time
import datetime

def R2(y_true, y_pred):
    import numpy
    return (numpy.corrcoef(y_true, y_pred)[0,1])**2

def date_2_year(date):
    return(date.year)

def datetojd(stddate): # Date to Julian day
    sdtdate = stddate.timetuple()
    jdate = sdtdate.tm_yday
    return(jdate)

In [ ]:
# import datasets

# Define the working directory path here
wd = "/path/to/working/directory"
dataset = pd.read_csv(wd+"Pusa_data.csv")
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset.insert(loc= 1, column= "Year", value= dataset['Date'].dt.year)
dataset.insert(loc= 2, column= "Jday", value= dataset['Date'].apply(datetojd))

# dropping the rows having NaN values
dataset = dataset.dropna()

# To reset the indices
dataset = dataset.reset_index(drop = True)

# Spliting Train and test set
train = dataset[dataset["Year"] < 2015] # Trainig set from 2010 to 2014
test = dataset[dataset["Year"] >= 2015] # Test set from 2015 to 2017

X_train = train[['T_min','T_max','T_mean','Ra', 'Rs']].values
X_test = test[['T_min','T_max','T_mean','Ra', 'Rs']].values
y_train = train[['ETo']].values
y_test = test[['ETo']].values

# Reshape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print("Shape of input training set: {}".format(X_train.shape))
print("Shape of target training set: {}".format(y_train.shape))
print("Shape of input test set: {}". format(X_test.shape))
print("Shape of target test set: {}".format(y_test.shape))

In [ ]:
# Define the transformer encoder
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6) (inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout) (x, x)
    x = layers.Dropout (dropout)(x)
    res = x + inputs
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu") (x)
    x = layers.Dropout(dropout) (x)
    x = layers.Conv1D(filters=inputs. shape[-1], kernel_size=1) (x)
    return x + res
    
# Build the transformer model
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    x = layers.GlobalAveragePooling1D(data_format="channels_first") (x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu") (x)
        x = layers.Dropout (mlp_dropout) (x)
    outputs = layers.Dense(1)(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(loss="mean_squared_error", optimizer=keras. optimizers.Adam(learning_rate=1e-4))
    return model

# Assuming X_train and y_train are already defined
# input_shape = X_train.shape[1:]

reg = KerasRegressor(
    model=build_model,
    # optimizer__learning_rate=0.001, # Removed optimizer settings here, let GridSearchCV handle it if needed
    model__input_shape=X_train.shape[1:],
    model__head_size=32,
    model__num_heads=4,
    model__ff_dim=256,
    model__num_transformer_blocks=4,
    model__mlp_units=[128],
    model__mlp_dropout=0.3,
    model__dropout=0.25,
    epochs=100,
    batch_size=16,
    random_state=123,
    verbose=0 # Set verbose to 0 to reduce output during grid search
)

params = {
    'model__head_size': [32,64],
    'model__num_heads': [4], # [4, 8]
    'model__ff_dim': [256, 512, 1024], 
    'model__num_transformer_blocks': [4], # [2, 4]
    'model__mlp_units': [[256], [512],], # [[64], [128], [256], [512], [128, 64], [256, 128]]
    'model__mlp_dropout': [0.3],
    'model__dropout': [0.25],
    'optimizer': ['adam'],  # Specify the optimizer name
    'optimizer__learning_rate': [1e-4], # Optimizer-specific parameter
    'batch_size': [16], # [16, 32, 64, 128]
    'epochs': [200]
}

regressor = GridSearchCV(estimator=reg, param_grid=params, n_jobs=10, cv=5, verbose=1,)# scoring="neg_mean_squared_error")

print("Starting GridSearchCV...")
regressor.fit(X_train, y_train)

# test result
y_pred = regressor.predict(X_test)

# train result
y_pred_train = regressor.predict(X_train)

# Preparing Observed and Predicted Test dataset
Test_res = pd.DataFrame({"Observed": y_test.ravel(),
                         "Predicted": y_pred.ravel()})

# Preparing Observed and Predicted Train dataset
Train_res = pd.DataFrame({"Observed": y_train.ravel(),
                          "Predicted": y_pred_train.ravel()})

# Test Plot

plt.scatter(Test_res.Observed, Test_res.Predicted, 
            c ="red", 
            linewidths = 0.5, 
            marker ="o", 
            edgecolor ="black", 
            s = 50)

plt.xlim(left= np.mean(Test_res.values)-3*np.std(Test_res.values),
         right= np.mean(Test_res.values)+3*np.std(Test_res.values))
plt.ylim(bottom= np.mean(Test_res.values)-3*np.std(Test_res.values),
         top= np.mean(Test_res.values)+3*np.std(Test_res.values))

plt.xlabel('Observed')
plt.ylabel('Predicted')
plt.text(np.mean(Test_res.values)-2.7*np.std(Test_res.values),
         np.mean(Test_res.values)+2.4*np.std(Test_res.values),  
         '$R^2$ = %0.3f' % R2(Test_res.Observed, Test_res.Predicted))

fig1 = plt.gcf()
fig1.set_size_inches(4.5, 4.5)
plt.show()
plt.draw()

# Train plot

plt.scatter(Train_res.Observed, Train_res.Predicted, 
            c ="blue", 
            linewidths = 0.5, 
            marker ="o", 
            edgecolor ="black", 
            s = 50)

plt.xlim(left= np.mean(Train_res.values)-3*np.std(Train_res.values),
         right= np.mean(Train_res.values)+3*np.std(Train_res.values))
plt.ylim(bottom= np.mean(Train_res.values)-3*np.std(Train_res.values),
         top= np.mean(Train_res.values)+3*np.std(Train_res.values))

plt.xlabel('Observed')
plt.ylabel('Predicted')
plt.text(np.mean(Train_res.values)-2.7*np.std(Train_res.values),
         np.mean(Train_res.values)+2.4*np.std(Train_res.values), 
         '$R^2$ = %0.3f' % R2(Train_res.Observed, Train_res.Predicted))

fig1 = plt.gcf()
fig1.set_size_inches(4.5, 4.5)
plt.show()
plt.draw()

# Export Test and Train dataset
Test_res.to_csv(wd+"Radiation_based_TNN_Test.csv", index = False, header=True)
Train_res.to_csv(wd+"Radiation_based_TNN_Train.csv", index = False, header=True)

# Best parameter
print(regressor.best_params_)
